In [ ]:
# necessary import 
from pyspark.sql import SparkSession
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit
from functools import reduce
# create a spark session
spark = SparkSession.builder.appName('DigitRecog').getOrCreate()
# loaded image
zero = ImageSchema.readImages("home/daniel/intership_supanut/numdata/0").withColumn("label", lit(0))
one = ImageSchema.readImages("home/daniel/intership_supanut/numdata/1").withColumn("label", lit(1))
two = ImageSchema.readImages("home/daniel/intership_supanut/numdata/2").withColumn("label", lit(2))
three = ImageSchema.readImages("home/daniel/intership_supanut/numdata/3").withColumn("label", lit(3))
four = ImageSchema.readImages("home/daniel/intership_supanut/numdata/4").withColumn("label", lit(4))
five = ImageSchema.readImages("home/daniel/intership_supanut/numdata/5").withColumn("label", lit(5))
six = ImageSchema.readImages("home/daniel/intership_supanut/numdata/6").withColumn("label", lit(6))
seven = ImageSchema.readImages("home/daniel/intership_supanut/numdata/7").withColumn("label", lit(7))
eight = ImageSchema.readImages("home/daniel/intership_supanut/numdata/8").withColumn("label", lit(8))
nine = ImageSchema.readImages("home/daniel/intership_supanut/numdata/9").withColumn("label", lit(9))
dataframes = [zero, one, two, three,four,
             five, six, seven, eight, nine]
# merge data frame
df = reduce(lambda first, second: first.union(second), dataframes)
# repartition dataframe 
df = df.repartition(200)
# split the data-frame
train, test = df.randomSplit([0.8, 0.2], 42)
df.printSchema()

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer
# model: InceptionV3
# extracting feature from images
featurizer = DeepImageFeaturizer(inputCol="image",
                                 outputCol="features",
                                 modelName="InceptionV3")
# used as a multi class classifier
lr = LogisticRegression(maxIter=5, regParam=0.03, 
                        elasticNetParam=0.5, labelCol="label")
# define a pipeline model
sparkdn = Pipeline(stages=[featurizer, lr])
spark_model = sparkdn.fit(train) # start fitting or training



from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# evaluate the model with test set
evaluator = MulticlassClassificationEvaluator() 
tx_test = spark_model.transform(test)
print('F1-Score ', evaluator.evaluate(tx_test, 
                                      {evaluator.metricName: 'f1'}))
print('Precision ', evaluator.evaluate(tx_test,
                                       {evaluator.metricName:                    'weightedPrecision'}))
print('Recall ', evaluator.evaluate(tx_test, 
                                    {evaluator.metricName: 'weightedRecall'}))
print('Accuracy ', evaluator.evaluate(tx_test, 
                                      {evaluator.metricName: 'accuracy'}))

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = false)
 |    |-- width: integer (nullable = false)
 |    |-- nChannels: integer (nullable = false)
 |    |-- mode: integer (nullable = false)
 |    |-- data: binary (nullable = false)
 |-- label: integer (nullable = false)

